<a href="https://colab.research.google.com/github/janinewhite/Personal-Expense-Tracker/blob/main/PersonalExpenseTracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import os

In [2]:
expensesFile = "expenses.csv"
budgetFile = "budget.txt"
expenses = []
budget = 0.0

In [3]:
# Save expenses to a CSV file
def save_expenses(filename=expensesFile) -> None:
    """
    Saves the current list of expenses to a CSV file.

    Args:
        filename (str, optional): The name of the file to save the expenses to.
                                  Defaults to "expenses.csv".
    """
    # Open the file in write mode, ensuring proper newline handling and UTF-8 encoding
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        # Define the field names for the CSV header
        fieldnames = ['date', 'category', 'amount', 'description']
        # Create a DictWriter object to write rows as dictionaries
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        # Write the header row to the CSV file
        writer.writeheader()
        # Write each expense dictionary as a row in the CSV file
        for expense in expenses:
            writer.writerow(expense)
    # Print a success message
    print("✅ Expenses saved.")

In [4]:
# Save budget to a text file
def save_budget(filename=budgetFile):
    """
    Saves the current budget to a TXT file.

    Args:
        filename (str, optional): The name of the file to save the budget to.
                                  Defaults to "budget.txt".
    """
    with open(filename, mode='w', encoding='utf-8') as file:
        file.write(str(budget))
    # Print a success message
    print("$ Budget saved.")

In [5]:
#Save files
def save_files() -> None:
    """
    Saves the current expenses and budget to their respective files.
    """
    save_expenses()
    save_budget()

In [6]:
# Load expenses from CSV
def load_expenses(filename=expensesFile) -> None:
    """
    Loads expenses from a CSV file into the global expenses list.

    Args:
        filename (str, optional): The name of the file to load expenses from.
    """
    # Check if the specified file exists
    if os.path.exists(filename):
        # Open the file in read mode with proper newline handling and UTF-8 encoding
        with open(filename, mode='r', newline='', encoding='utf-8') as file:
            # Create a DictReader object to read rows as dictionaries
            reader = csv.DictReader(file)
            # Iterate over each row in the CSV file
            for row in reader:
                try:
                    # Append the expense data to the global expenses list
                    expenses.append({
                        'date': row['date'],
                        'category': row['category'],
                        'amount': float(row['amount']), # Convert amount to float
                        'description': row['description']
                    })
                except (ValueError, KeyError):
                    # Handle cases where the row has missing fields or invalid amount format
                    print("⚠️ Skipping invalid entry in file.")
    else:
        # Print a message if the file does not exist
        print(f"ℹ️ File '{filename}' not found. Starting with an empty expense list.")

In [7]:
# Load budget from TXT
def load_budget(filename=budgetFile) -> None:
    """
    Loads the budget from a TXT file into the global budget variable.

    Args:
        filename (str, optional): The name of the file to load the budget from.
                                  Defaults to "budget.txt".
    """
    global budget  # Declare budget as global to modify the global variable
    if os.path.exists(filename):
        with open(filename, mode='r', encoding='utf-8') as file:
            try:
                budget = float(file.read())
                print("💲 Budget loaded successfully.")
            except ValueError:
                print("⚠️ Invalid budget format in file. Budget not loaded.")
    else:
        print(f"ℹ️ Budget file '{filename}' not found. Starting with a budget of $0.00.")
        budget = 0.0 # Initialize budget to 0 if file not found

In [8]:
# Load expenses and budget
def load_files() -> None:
    """
    Loads expenses and budget from their respective files.
    """
    load_expenses()
    load_budget()

In [9]:
# Add a new expense
def add_expense() -> None:
    """
    Prompts the user to enter details for a new expense and adds it to the global expenses list.
    Handles invalid amount input.
    """
    # Get expense details from user input
    date = input("Enter date (YYYY-MM-DD): ")
    category = input("Enter category (e.g., Food, Travel): ")
    try:
        # Attempt to convert amount input to a float
        amount = float(input("Enter amount spent: "))
    except ValueError:
        # Handle invalid amount input
        print("❌ Invalid amount. Expense not added.")
        return # Exit the function if amount is invalid

    description = input("Enter a brief description: ")

    # Create a dictionary for the new expense
    expense = {
        'date': date,
        'category': category,
        'amount': amount,
        'description': description
    }
    # Append the new expense to the global expenses list
    expenses.append(expense)
    print("✅ Expense added.")

In [10]:
# Delete Expense
def delete_expense() -> None:
    """
    Displays the current list of expenses and allows the user to select and delete one.
    Saves the updated expense list after deletion.
    Handles invalid input for selection.
    """
    print("\n🗂️ Select an expense to delete:")
    # Enumerate through the expenses list to display each expense with its index
    for i, expense in enumerate(expenses, start=1):
        # Print the expense details with formatting
        print(f"{i}. {expense['date']} | {expense['category']} | ${expense['amount']:.2f} | {expense['description']}")
    try:
        # Get the index of the expense to delete from user input
        index = int(input("Enter the number of the expense to delete: "))
        # Check if the entered index is valid
        if 1 <= index <= len(expenses):
            # Remove the selected expense from the list
            removed = expenses.pop(index - 1)
            # Print a success message with details of the deleted expense
            print(f"✅ Deleted: {removed['date']} | {removed['category']} | ${removed['amount']:.2f}")
            # Save the updated expenses list to the file
            save_expenses()
            # Display the updated list of expenses
            view_expenses()
        else:
            # Handle invalid index selection
            print("❌ Invalid selection.")
    except ValueError:
        # Handle non-integer input for selection
        print("❌ Please enter a valid number.")

In [11]:
def save_return() -> None:
    # Save expenses then return to main menu
    save_expenses()
    menu()

In [12]:
def view_expenses() -> None:
    """
    Display all recorded expenses and provide options to delete an expense or return to the main menu.

    If no expenses are recorded, a message is displayed. Otherwise, all expenses are listed
    with their details (date, category, amount, description).

    Returns:
        None
    """
    if not expenses:
        print("📭 No expenses recorded.")
        return

    print("\n📋 Your Expenses:")
    for i, expense in enumerate(expenses, start=1):
        # Check if expense has all required fields
        required_fields = ['date', 'category', 'amount', 'description']
        if all(field in expense for field in required_fields):
            print(f"{i}. {expense['date']} | {expense['category']} | "
                  f"${expense['amount']:.2f} | {expense['description']}")
        else:
            print(f"{i}. ⚠️ Incomplete expense entry.")

    # Display menu options
    print("\n📌 Expenses Menu:")
    print("1. Delete expense")
    print("2. Return to main menu")

    # Handle user choice
    choice = input("Choose an option (1-2): ")

    if choice == '1':
        delete_expense()
    elif choice == '2':
        menu()
    else:
        print("❌ Invalid choice. Please try again.")

In [13]:
# Set monthly budget
def set_budget() -> None:
    """
    Prompts the user to enter their monthly budget and updates the global budget variable.
    Handles invalid input for the budget amount.
    Saves the updated budget after setting it.
    """
    global budget  # Declare budget as global to modify the global variable
    try:
        # Get budget amount from user input and convert to float
        budget = float(input("Enter your monthly budget: "))
        # Print a success message with the formatted budget amount
        print(f"✅ Budget set to ${budget:.2f}")
        # Save the updated budget to the file
        save_budget()
    except ValueError:
        # Handle invalid budget input
        print("❌ Invalid budget amount.")

In [14]:
# Track budget
def track_budget() -> None:
    """
    Calculates and displays the total amount spent and compares it to the budget.
    """
    # Calculate the total amount spent by summing up the 'amount' from each expense
    total_spent = sum(exp['amount'] for exp in expenses)
    print(f"\n💰 Total spent: ${total_spent:.2f}")

    # Check if a budget has been set
    if budget == 0:
        print("⚠️ No budget set.")
    # Check if the total spent exceeds the budget
    elif total_spent > budget:
        print("🚨 You have exceeded your budget!")
    # Otherwise, calculate and display the remaining budget
    else:
        print(f"🟢 You have ${budget - total_spent:.2f} left for the month.")

In [15]:
def budget_menu() -> None:
    """
    Display and handle the budget management menu.

    This function shows the current budget, presents options to set a new budget
    or return to the main menu, and processes the user's selection.
    """
    track_budget()
    # Define menu options
    menu_options = {
        '1': ('Set budget', set_budget),
        '2': ('Return to main menu', save_return),
    }
    while True:
        # Display menu options
        print("\n📌 Budget Menu:")
        for key, (description, _) in menu_options.items():
            print(f"{key}. {description}")
        choice = input(f"Choose an option (1-{len(menu_options)}): ")

        if choice in menu_options:
            # Execute the function associated with the chosen option
            _, function = menu_options[choice]
            function()
            if choice == '2':
                break
        else:
            print("❌ Invalid choice. Please try again.")

In [16]:
def menu() -> None:
    """
    Display an interactive menu for the expense tracker application.
    This function loads expenses from storage and presents a menu with options
    to add, view, and track expenses, as well as save data and exit the application.
    """
    load_files()
    # Define menu options as a dictionary for better maintainability
    menu_options = {
        '1': ('Add expense', add_expense),
        '2': ('View expenses', view_expenses),
        '3': ('Track budget', budget_menu),
        '4': ('Save expenses and budget', save_files),
        '5': ('Save expenses and budget then exit', save_files),
        '6': ('Exit (no save)', None)
    }
    while True:
        # Display menu
        print("\n📌 Expense Tracker Menu:")
        for key, (description, _) in menu_options.items():
            print(f"{key}. {description}")

        choice = input(f"Choose an option (1-{len(menu_options)}): ")

        if choice in menu_options:
            if choice != '6':
                # Execute the function associated with the chosen option
                _, function = menu_options[choice]
                function()
            if choice == '5' or choice =='6':
                print("👋 Goodbye!")
                break
        else:
            print("❌ Invalid choice. Please try again.")

In [17]:
# Run the program
if __name__ == "__main__":
    menu()

ℹ️ File 'expenses.csv' not found. Starting with an empty expense list.
ℹ️ Budget file 'budget.txt' not found. Starting with a budget of $0.00.

📌 Expense Tracker Menu:
1. Add expense
2. View expenses
3. Track budget
4. Save expenses
5. Save expenses and exit
6. Exit (no save)
Choose an option (1-6): 5
✅ Expenses saved.
👋 Goodbye!
